In [1]:
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

Using TensorFlow backend.


In [2]:
from keras.applications import vgg19
from keras import backend as K

In [3]:
base_image_path = 'saak.jpg'
style_image_path = 'style.jpg'
iterations = 100
total_variation_weight = 100.0
content_weight = 5.0
style_weight = 500.0

In [4]:
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [5]:
def preprocess_image(image_path):
    img = load_img(base_image_path,target_size=(img_nrows,img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [6]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3,img_nrows,img_ncols))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((img_nrows,img_ncols,3))
    
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
#     BGR to RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
base_image = K.variable(preprocess_image(base_image_path))
style_image = K.variable(preprocess_image(style_image_path))

In [8]:
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1,3,img_nrows,img_ncols))
else:
    combination_image = K.placeholder((1,img_nrows,img_ncols,3))

In [9]:
input_tensor = K.concatenate([base_image,style_image,combination_image],axis=0)
input_tensor

<tf.Tensor 'concat:0' shape=(3, 400, 225, 3) dtype=float32>

In [10]:
model = vgg19.VGG19(input_tensor = input_tensor,weights = 'imagenet', include_top = False)
print ('Model loaded')

Model loaded


In [11]:
model.layers

In [12]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [13]:
outputs_dict

{'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 400, 225, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 400, 225, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 200, 112, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 200, 112, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 200, 112, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 100, 56, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 100, 56, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 100, 56, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 100, 56, 256) dtype=float32>,
 'block3_conv4': <tf.Tensor 'block3_conv4/Relu:0' shape=(3, 100, 56, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(3, 50, 28, 256) dtype=float32>,
 'bloc

In [14]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x,(2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [15]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C))/(4.*(channels ** 2) * (size ** 2))

In [16]:
def content_loss(base,combination):
    return K.sum(K.square(combination - base))

In [17]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [18]:
loss = K.variable(0.)
layer_features = outputs_dict['block5_conv2']
layer_features

<tf.Tensor 'block5_conv2/Relu:0' shape=(3, 25, 14, 512) dtype=float32>

In [19]:
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features, combination_features)

In [20]:
feature_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * s1
loss += total_variation_weight * total_variation_loss(combination_image)

In [21]:
grads = K.gradients(loss, combination_image)

In [22]:
outputs = [loss]

In [23]:
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

In [24]:
f_outputs = K.function([combination_image], outputs)

In [25]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten.astype('float64')
    return loss_value, grad_values

In [26]:
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grads_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grads_values)
        self.loss_value = None
        self.grads_values = None
        return grad_values

In [27]:
evaluator = Evaluator()

In [28]:
x = preprocess_image(base_image_path)

In [29]:
for i in xrange(iterations):
    print ("Start of iteration ", i)
    start_time = time.time()
    print (start_time)
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime = evaluator.grads, maxfun = 20)
    print('Current loss value ', min_val)
    img = deprocess_image(x.copy())
    fname = 'iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as ', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration  0
1500227133.81
Current loss value  2.59753e+10
Image saved as  iteration_0.png
Iteration 0 completed in 296s
Start of iteration  1
1500227429.82
Current loss value  2.29153e+10
Image saved as  iteration_1.png
Iteration 1 completed in 273s
Start of iteration  2
1500227703.38
Current loss value  2.14436e+10
Image saved as  iteration_2.png
Iteration 2 completed in 272s
Start of iteration  3
1500227975.8
Current loss value  2.05545e+10
Image saved as  iteration_3.png
Iteration 3 completed in 272s
Start of iteration  4
1500228248.41
Current loss value  1.97091e+10
Image saved as  iteration_4.png
Iteration 4 completed in 273s
Start of iteration  5
1500228521.42
Current loss value  1.91542e+10
Image saved as  iteration_5.png
Iteration 5 completed in 271s
Start of iteration  6
1500228793.3
Current loss value  1.87225e+10
Image saved as  iteration_6.png
Iteration 6 completed in 271s
Start of iteration  7
1500229065.26
Current loss value  1.83922e+10
Image saved as  iteratio

;iuywsX a# 